## エクセルデータを少しだけ綺麗にする

In [1]:
import pandas as pd

In [2]:
_df = pd.read_excel('../20200708_fencing competiton.xlsx')
_df.head()

,competition,name of game(fencer_NOC-fencer_NOC),score
0,Vancouver WA,ALEXANIN Dmitriy_KAZ-SONG Jae Ho_KOR,9-15
1,Vancouver WA,BAYARD Alexis_SUI-PARK Sangyoung_KOR,12-13
2,Vancouver WA,BERTA Daniel_HUN-GAROZZO Enrico_ITA,8-15
3,Vancouver WA,BIDA Sergey_RUS-PRIINITS Sten_EST,15-13
4,Vancouver WA,BIDA Sergey_RUS-SONG Jae Ho_KOR,15-10


In [3]:
player_info = _df['name of game(fencer_NOC-fencer_NOC)'].str.split('-').items()
score_info = _df['score'].str.split('-').items()

df = pd.concat([
    _df,
    pd.DataFrame([(*x[0].split('_'), *x[1].split('_')) for _, x in player_info], 
                 columns=['p1_name', 'p1_nat', 'p2_name','p2_nat']),
    pd.DataFrame([x for _, x in score_info],
                 columns=['p1_score', 'p2_score'])
], axis=1)
df.index = df.index.rename('Game_ID')
df.head()

,competition,name of game(fencer_NOC-fencer_NOC),score,p1_name,p1_nat,p2_name,p2_nat,p1_score,p2_score
Game_ID,,,,,,,,,
0,Vancouver WA,ALEXANIN Dmitriy_KAZ-SONG Jae Ho_KOR,9-15,ALEXANIN Dmitriy,KAZ,SONG Jae Ho,KOR,9,15
1,Vancouver WA,BAYARD Alexis_SUI-PARK Sangyoung_KOR,12-13,BAYARD Alexis,SUI,PARK Sangyoung,KOR,12,13
2,Vancouver WA,BERTA Daniel_HUN-GAROZZO Enrico_ITA,8-15,BERTA Daniel,HUN,GAROZZO Enrico,ITA,8,15
3,Vancouver WA,BIDA Sergey_RUS-PRIINITS Sten_EST,15-13,BIDA Sergey,RUS,PRIINITS Sten,EST,15,13
4,Vancouver WA,BIDA Sergey_RUS-SONG Jae Ho_KOR,15-10,BIDA Sergey,RUS,SONG Jae Ho,KOR,15,10


In [4]:
## Competitionってなに

df['competition'].unique()

array(['Vancouver WA', 'Budapest GP', 'Heidenheim WA'], dtype=object)

## 国別・選手別で試合データのばらつきを見てみる

In [5]:
import matplotlib.pyplot as plt

ply_cnts = pd.concat([df['p1_name'],df['p2_name']]).value_counts()
nat_cnts = pd.concat([df['p1_nat'],df['p2_nat']]).value_counts()

plt.figure(figsize=(20,4))
plt.bar(ply_cnts.index, ply_cnts.values)
plt.xticks(rotation='vertical')
plt.title('Player Frequency')
plt.ylabel('Games Played')
plt.xlabel('Name')
plt.show()

plt.figure(figsize=(20,4))
plt.bar(nat_cnts.index, nat_cnts.values)
plt.xticks(rotation='vertical')
plt.title('Nationality Frequency')
plt.ylabel('Games Played')
plt.xlabel('Nation')
plt.show()

<Figure size 2000x400 with 1 Axes>

<Figure size 2000x400 with 1 Axes>

## 得点を見てみる

In [6]:
import seaborn as sns

scores = df[['p1_score','p2_score']].astype(int).values

data = {
    'all scores': scores.flatten(),
    'lose scores':np.sort(scores).T[0], 
    'win scores':np.sort(scores).T[1],
    'diff': win_scores - lose_scores
}

fig, axs = plt.subplots(2,2,figsize=(12, 8))
for i, (name, values) in enumerate(data.items()):
    ax = axs[i//2, i%2]
    ax.set_title(name)
    sns.histplot(values,bins=15,ax=ax)
plt.show()

NameError: name 'np' is not defined

必ずしも15点を取って勝っているわけではない！

## 追加データ

FIE websiteからデータを引っ張ってくることで対戦成績が分かるのでは？という狙い
必要あれば、スクレイピングするのはあり。（APIないので面倒）

In [ ]:
import requests
from bs4 import BeautifulSoup

fie_df = pd.read_excel('../2020_10_09_fie.xlsx')
url = fie_df.iloc[0]['url']
print(url)
r = requests.get(url)

In [ ]:
soup = BeautifulSoup(r.text, 'html.parser')
mydivs = soup.find("div", {"class": "AthleteBio-sidebar SecondaryMenu-block js-secondary-menu-item"})
mydivs.find("div", {"class": "Tabs-itemsWrapper"}).findAll("div", {"class":"Tabs-item"})